In [2]:
import os
import numpy as np
import tifffile
import napari

from scipy.io import loadmat
from skimage import transform

from waveorder import waveorder_microscopy, wavelet_softThreshold
from waveorder.io import WaveorderReader

# Table of contents
- View dataset
- Load data
    - Load raw images
    - Load calibration data
    - Load background images
- Recostruct label-free channels
    - Register images
        - Crop edges
        - View registered images
    - Reconstruct Stokes images
        - Initialize reconstructor
        - Reconstruct Raw Stokes images
            - View raw Stokes images
        - Denoise S0 images
            - Compare raw and denoised images
        - Normalize Stokes images
            - View normalized Stokes images and background images
        - Correct background
            - View background corrected Stokes images
        - Denoise S1 and S2 images
            - View denoised Stokes images
    - Compute transmission, phase, retardance, and orientation
- View reconstruction results

# View dataset

In [3]:
# assume data is in ~/Downloads folder
data_dir = os.path.join(os.path.expanduser('~'), 'Downloads')
data_path = os.path.join(data_dir, 'miPolScope_fig3_cardiomyocytes_labelfree.zarr')

# Check that data path exists
if not os.path.exists(data_path):
    raise ValueError('Data path does not exist.')

In [4]:
viewer = napari.Viewer()
layers = viewer.open(os.path.join(data_path, 'Row_0/Col_0/Pos_000'), plugin='napari-ome-zarr')

version mismatch: detected:FormatV01, requested:FormatV03
version mismatch: detected:FormatV03, requested:FormatV01


# Load data

## Load raw images

In [5]:
wo_data = WaveorderReader(data_path, data_type='zarr')
I = wo_data.get_zarr(0)
n_timepoints, n_channels, n_slices, *img_size = I.shape

In [6]:
print(
    f'Number of time points: {n_timepoints}\n'
    f'Number of channels: {n_channels}\n'
    f'Number of slices: {n_slices}\n'
    f'Image size: {img_size}'
)

Number of time points: 95
Number of channels: 4
Number of slices: 20
Image size: [514, 616]


As demonstration, we will analyze only the first 5 timepoints

In [7]:
n_timepoints = 5

In [8]:
# load data into memory
I = np.array(I[:n_timepoints])

## Load calibration data

In [9]:
cal_data = loadmat(os.path.join(data_path,'calibration.mat'))

In [10]:
A = np.transpose(cal_data['A'].astype('float'), (2, 3, 0, 1)) # A has shape (size_Y, size_X, N_channels, N_Stokes)
black_level = cal_data['black_level'][0][0].astype('uint16')

tform0 = transform.AffineTransform(cal_data['tform0'].T)
tform45 = transform.AffineTransform(cal_data['tform45'].T)
tform90 = transform.AffineTransform(cal_data['tform90'].T)
tform135 = transform.AffineTransform(cal_data['tform135'].T)

## Load background images

In [11]:
S_bg = tifffile.imread(os.path.join(data_path, 'Stokes_bg.ome.tif'))

# Recostruct label-free channels

## Register images

In [12]:
I_registered = np.zeros((n_timepoints, 4, n_slices, *img_size), dtype='float')

for t in range(n_timepoints):
    for c, tform in enumerate((tform0, tform45, tform90, tform135)):
        for z in range(n_slices):
            I_registered[t,c,z] = transform.warp(I[t,c,z], tform.inverse, preserve_range=True)

### Crop edges

In [13]:
I_registered = I_registered[..., 7:-7, 13:-13]
img_size = I_registered.shape[-2:]

### View registered images

In [14]:
viewer = napari.view_image(I_registered)

## Reconstruct Stokes images

### Initialize reconstructor

In [15]:
wavelength = 525 # in nm
NA_obj = 1.2 # Numerical Aperture of Objective
NA_illu = 0.4 # Numerical Aperture of Condenser
n_objective_media = 1.33 # refractive index of objective immersion media
mag = 30 # effective magnification
n_slices = I.shape[-3] # number of slices in z-stack
z_step_um = 0.25 # z-step size in um
pad_z = 5 # slices to pad for phase reconstruction boundary artifacts
pixel_size_um = 3.45 # camera pixel size in um
bg_correction = 'local_fit' # BG correction method: 'None', 'local_fit', 'global'
mode = '3D' # phase reconstruction mode, '2D' or '3D'
use_gpu = False
gpu_id = 0 

In [16]:
z_defocus = -(np.r_[:n_slices] - n_slices // 2) * z_step_um # assumes stack starts from the bottom
swing = 0
ps = pixel_size_um / mag

reconstructor = waveorder_microscopy(img_dim=img_size,
                                     lambda_illu=wavelength/1000,
                                     ps=ps,
                                     NA_obj=NA_obj,
                                     NA_illu=NA_illu,
                                     z_defocus=z_defocus,
                                     chi=swing,
                                     n_media=n_objective_media,
                                     cali=True,
                                     bg_option=bg_correction,
                                     A_matrix=A,
                                     QLIPP_birefringence_only=False,
                                     pad_z=pad_z,
                                     phase_deconv=mode,
                                     illu_mode='BF',
                                     use_gpu=use_gpu,
                                     gpu_id=gpu_id)

### Reconstruct Raw Stokes images

In [17]:
S_raw = np.zeros((n_timepoints, 3, n_slices, *img_size), dtype='float')

for t in range(n_timepoints):
    S_raw_ = reconstructor.Stokes_recon(np.moveaxis(I_registered[t], 1, -1) - black_level)
    S_raw[t] = np.moveaxis(S_raw_, -1, 1)

#### View raw Stokes images

In [18]:
viewer = napari.Viewer()
viewer.add_image(S_raw[:,0], name='S0_raw', colormap='gray')
viewer.add_image(S_raw[:,1], name='S1_raw', colormap='RdBu', visible=False)
viewer.add_image(S_raw[:,2], name='S2_raw', colormap='RdBu', visible=False)

<Image layer 'S2_raw' at 0x7f9220c4c640>

### Denoise S0 images

In [19]:
S0_raw_denoised = wavelet_softThreshold(S_raw[:,0], 'db8', 200, level=2, axes=(1,2,3))
S0_raw_denoised = np.expand_dims(S0_raw_denoised, axis=1)

/Users/ivan.ivanov/anaconda3/envs/mipolscope/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 2 is too high: all coefficients will experience boundary effects.
  warnings.warn(


#### Compare raw and denoised images

In [20]:
viewer = napari.Viewer()
viewer.add_image(S_raw[:,0], name='S0_raw', colormap='gray')
viewer.add_image(S0_raw_denoised[:,0], name='S0_raw_denoised', colormap='gray')

<Image layer 'S0_raw_denoised' at 0x7f926a3f02e0>

### Normalize Stokes images

In [21]:
S_norm = np.zeros_like(S_raw)

for t in range(n_timepoints):
    for z in range(n_slices):
        S_norm[t,:,z] = reconstructor.Stokes_transform(np.concatenate([S0_raw_denoised[t,:,z], 
                                                                       S_raw[t,1:,z]]))

#### View normalized Stokes images and background images

In [22]:
viewer = napari.Viewer()
viewer.add_image(S_norm[:,0], name='S0_norm', colormap='gray')
viewer.add_image(S_norm[:,1], name='S1_norm', colormap='RdBu', visible=False, contrast_limits=(-0.2, 0.2))
viewer.add_image(S_norm[:,2], name='S2_norm', colormap='RdBu', visible=False, contrast_limits=(-0.2, 0.2))

viewer.add_image(S_bg[0], name='S0_bg', colormap='gray', visible=False, contrast_limits=viewer.layers[0].contrast_limits)
viewer.add_image(S_bg[1], name='S1_bg', colormap='RdBu', visible=False, contrast_limits=(-0.2, 0.2))
viewer.add_image(S_bg[2], name='S2_bg', colormap='RdBu', visible=False, contrast_limits=(-0.2, 0.2))

<Image layer 'S2_bg' at 0x7f92112b7490>

### Correct background

In [23]:
S_corr = np.zeros_like(S_raw)

for t in range(n_timepoints):
    S_corr_ = reconstructor.Polscope_bg_correction(np.moveaxis(S_norm[t], 1, -1), S_bg)
    S_corr[t] = np.moveaxis(S_corr_, -1, 1)

#### View background corrected Stokes images

In [24]:
viewer = napari.Viewer()
viewer.add_image(S_corr[:,0], name='S0_corr', colormap='gray')
viewer.add_image(S_corr[:,1], name='S1_corr', colormap='RdBu', visible=False, contrast_limits=(-0.05, 0.05))
viewer.add_image(S_corr[:,2], name='S2_corr', colormap='RdBu', visible=False, contrast_limits=(-0.05, 0.05))

<Image layer 'S2_corr' at 0x7f9210f640a0>

### Denoise S1 and S2 images

In [25]:
# wavelet denoising
S1_denoised = wavelet_softThreshold(S_corr[:, 1], 'db8', 1e-2, level=2, axes=(1,2,3))
S2_denoised = wavelet_softThreshold(S_corr[:, 2], 'db8', 1e-2, level=2, axes=(1,2,3))
S_denoised = np.stack((S_corr[:, 0], S1_denoised, S2_denoised), axis=1)

/Users/ivan.ivanov/anaconda3/envs/mipolscope/lib/python3.9/site-packages/pywt/_multilevel.py:43: UserWarning: Level value of 2 is too high: all coefficients will experience boundary effects.
  warnings.warn(


#### View denoised Stokes images

In [26]:
viewer = napari.Viewer()
viewer.add_image(S_denoised[:,0], name='S0', colormap='gray')
viewer.add_image(S_denoised[:,1], name='S1', colormap='RdBu', visible=False, contrast_limits=(-0.05, 0.05))
viewer.add_image(S_denoised[:,2], name='S2', colormap='RdBu', visible=False, contrast_limits=(-0.05, 0.05))

<Image layer 'S2' at 0x7f921c4d9e50>

## Compute transmission, phase, retardance, and orientation

In [27]:
retardance = np.zeros((n_timepoints, 1, n_slices, *img_size))
orientation = np.zeros_like(retardance)
transmission = np.zeros_like(retardance)

for t in range(n_timepoints):
    phys_props_ = reconstructor.Polarization_recon(np.moveaxis(S_denoised[t], 1, -1))
    retardance[t,0], orientation[t,0], transmission[t,0] = np.moveaxis(phys_props_, -1, 1)
    
transmission /= transmission[0].mean()

In [28]:
phase = np.zeros((n_timepoints, 1, n_slices, *img_size))

for t in range(n_timepoints):
    phase_ = reconstructor.Phase_recon_3D(np.moveaxis(S_corr[t, 0], 0, -1), method='Tikhonov')
    phase[t,0] = np.moveaxis(phase_,-1,0)

# View reconstruction results

In [29]:
viewer = napari.Viewer()
viewer.add_image(retardance, name='retardance', colormap='gray', contrast_limits=(0, 0.03))
viewer.add_image(orientation, name='orientation', colormap='gray', visible=False, contrast_limits=(0, np.pi))
viewer.add_image(transmission, name='transmission', colormap='gray', visible=False, contrast_limits=(0.8, 1.2))
viewer.add_image(phase, name='phase', colormap='gray', visible=False, contrast_limits=(-0.03, 0.03))

<Image layer 'phase' at 0x7f926bc7dee0>